# Introduction

this notebook demos example of using llm in a MPS backend (apple silicon GPU) using torch 2.x to do a local function calling

Referece:
* https://ollama.com/blog/tool-support
* torch 2.x MPS Backend: https://pytorch.org/docs/stable/notes/mps.html
* https://blog.pamelafox.org/2024/08/making-ollama-compatible-rag-app.html
* https://platform.openai.com/docs/guides/function-calling

ollama server log on mac `tail -f $HOME/.ollama/logs/server.log`

In [1]:
import os
import torch
import applyllm as apl
# from dotenv import load_dotenv
from dotenv import dotenv_values

print(apl.__version__)

0.0.7


In [2]:
# check that MPS is availabe (Metal Performance Shaders)
if not torch.backends.mps.is_available():
    print("MPS is not available")
else:
    print("MPS is available")
    mps_device = torch.device("mps")
    print(mps_device)



MPS is available
mps


## Define global variables

In [3]:
# backend = "ollama3.2"
backend = "mistral-nemo"
# backend = "mistral-nemo-inst"
# backend = "ollama-inst"
# backend = "lmstudio"
# backend = "openwebui"
api_key="EMPTY"
sys_msg={}
# nemo need sys message
# nemo works with ollama 0.3.14 version, not the 0.3.13 version
# https://github.com/ollama/ollama/issues/6713#issuecomment-2338933604

# both the ollama3.1:8b and llama3.1:8b-instruct-q3_K_M models
# works with openai compatible function calling on device
if backend == "ollama3.2":
    # model="gorilla-openfunctions-v2-q4_K_M"
    model="llama3.2:3b"
    # model="llama3.1:8b"
    endpoint="http://localhost:11434/v1"
elif backend == "ollama-inst":
    model="llama3.1:8b-instruct-q3_K_M"
    endpoint="http://localhost:11434/v1"
elif backend == "lmstudio":
    model="TheBloke/gorilla-openfunctions-v1-GGUF"
    endpoint="http://localhost:1234/v1"
elif backend == "mistral-nemo-inst":
    model="mistral-nemo:12b-instruct-2407-fp16"
    endpoint="http://localhost:11434/v1"
elif backend == "mistral-nemo":
    model="mistral-nemo:12b"
    endpoint="http://localhost:11434/v1"
    # sys_msg={ 
    #     "role": "system", 
    #     "content": "You are a helpful AI with tool calling capabilities. Watch out for the 'tools' element in the request."
    #     # "content": "You are a helpful AI with tool calling capabilities."
    # }
elif backend == "openwebui":
    # notice the openwebui endpoint is not just pass through forward to ollama
    # the current openwebui endpoint is not openai compatible
    model="llama3.1:70b"
    endpoint="https://core-llmtest.med.uni-muenchen.de/ollama/v1"
    config = dotenv_values(dotenv_path="envs/openwebui.env")
    api_key = config["API_KEY"]

In [4]:
# from openai import OpenAI

# client = OpenAI(
#     base_url = endpoint,
#     api_key=api_key, # required, but unused
# )

# response = client.chat.completions.create(
#   model=model,
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Who won the world series in 2020?"},
#     {"role": "assistant", "content": "The LA Dodgers won in 2020."},
#     {"role": "user", "content": "Where was it played?"}
#   ]
# )
# print(response.choices[0].message.content)

In [5]:
from openai import OpenAI
import json

def get_llm_response(
    prompt="Call me an Uber ride type \"Plus\" in Berkeley at zipcode 94704 in 10 minutes", 
    model="gorilla-openfunctions-v2-q4_K_M",
    endpoint="http://localhost:11434/v1",
    api_key=api_key,
    sys_msg=sys_msg,
    functions=[]):


    client = OpenAI(
        api_key=api_key,
        base_url=endpoint,
    )
    # client = CustomOpenAIClient(
    #     api_key="EMPTY",
    #     base_url=endpoint
    # )
  
    try:
        # extend with custom prompt
        # prompt = f'<<question>> {prompt} <<function>> {json.dumps(functions)}'
        completion = client.chat.completions.create(
            model=model, # model from the ollama list
            temperature=0.01,
            top_p=1,
            max_tokens=80,
            messages=[{"role": "user", "content": prompt}, ] if sys_msg == {} else [sys_msg, {"role": "user", "content": prompt}, ],
            tools=functions,
            # tool_choice= "required",
            stream=False,
            tool_choice= "auto",
        )
        print(completion)
        return completion.choices[0]
    except Exception as e:
        print(e, model, prompt)

### Example of Parallel Function calling
Human: 
```text
What's the weather like in the two cities of Boston and San Francisco?
```
Agent: 
```json
...
        "tool_calls": [
          {
            "id": "1",
            "function": {
              "arguments": "{\"location\": \"Boston, MA\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          },
          {
            "id": "1",
            "function": {
              "arguments": "{\"location\": \"San Francisco, CA\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
```

In [6]:
# response = get_llm_response("hi", functions=[], model=model, endpoint=endpoint, api_key=api_key)
# print(response)

In [7]:
query = "What's the weather like in the two cities of Boston and San Francisco?"
functions = [
    {   "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]

# from pydantic import BaseModel
# import openai
# class get_current_weather(BaseModel):
#     """Get the current weather in a given location"""
#     location: str # The city and state, e.g. San Francisco, CA
#     unit: str

# functions = [openai.pydantic_function_tool(get_current_weather)]

response = get_llm_response(query, functions=functions, model=model, endpoint=endpoint, api_key=api_key)

ChatCompletion(id='chatcmpl-576', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4ukqlret', function=Function(arguments='{"location":"Boston, MA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_r3egd4vg', function=Function(arguments='{"location":"San Francisco, CA"}', name='get_current_weather'), type='function')]))], created=1731265636, model='mistral-nemo:12b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=46, prompt_tokens=102, total_tokens=148, completion_tokens_details=None, prompt_tokens_details=None))


In [8]:
# successfully make parallel function calling
if model == "gorilla-openfunctions-v2-q4_K_M":
    from utils.openai import CustomLLMResponseAdapter
    completion = CustomLLMResponseAdapter.adapt_response(response=response.message.content)
    response = completion.choices[0]

if response is not None:
    print(response.message.content)
else:
    print("No response")

In [9]:
if response is not None:
    openai_json_response = response.to_json()
    print(openai_json_response)

{
  "finish_reason": "tool_calls",
  "index": 0,
  "message": {
    "content": "",
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_4ukqlret",
        "function": {
          "arguments": "{\"location\":\"Boston, MA\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      },
      {
        "id": "call_r3egd4vg",
        "function": {
          "arguments": "{\"location\":\"San Francisco, CA\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  }
}
